In [1]:
'''
Adapated from: https://github.com/CVxTz/kinship_prediction/blob/master/code/vgg_face.py 
to parse the images and data
This is NOT used to make the model! 
Additionally, this is run on a Windows OS. Some code may not be need if it is on a UNIX or LINUX OS. 
'''
import pandas as pd
from glob import glob #for finding files recursively 
from collections import defaultdict

In [2]:
relationshipsCSV = "./SMILE/train_relationships.csv"
train_images_folder = "./SMILE/train/"
#what is the difference between train and train-face?
val_set = "F09" #can change this, this is randomly generated 
'''
Idea of hypertuning:
using different validation sets for each iteration 
maybe instead of using only only family as the validation, we can use a group
or change the validation set for each iteration of training the model 
'''

'\nIdea of hypertuning:\nusing different validation sets for each iteration \nmaybe instead of using only only family as the validation, we can use a group\nor change the validation set for each iteration of training the model \n'

In [3]:
availble_images = glob(train_images_folder + "*/*/*.jpg")
'''
Only for windows pc:
'''
for a in range(0, len(availble_images)):
    availble_images[a] = availble_images[a].replace("\\", "/")   
    
all_ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in availble_images] #all the people

In [4]:
#creating the training set
train_images = [x for x in availble_images if val_set not in x]
train_person_to_images_map = defaultdict(list)
for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

#creating the validation set
val_images = [x for x in availble_images if val_set in x]
val_person_to_images_map = defaultdict(list)
for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

In [ ]:
relationships = pd.read_csv(relationshipsCSV)
# Create a list of tuples
relationship_pairs = [(row['p1'], row['p2']) for index, row in relationships.iterrows()]
relationship_pairs = [x for x in relationship_pairs if x[0] in all_ppl and x[1] in all_ppl] #data cleaning

In [ ]:
train = [x for x in relationship_pairs if val_set not in x[0]]
val = [x for x in relationship_pairs if val_set in x[0]]
#split the train and validation